In [6]:
# ========== Functions =============

import requests
import datetime as dt
import xml.etree.ElementTree as ET
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import colorbar, colors

def get_param_names(url):
    """ Get parameters metadata"""
    req = requests.get(url)
    params = {}
    
    if req.status_code == 200:
        xmlstring = req.content
        tree = ET.ElementTree(ET.fromstring(xmlstring))      
        for p in tree.iter(tag='{http://inspire.ec.europa.eu/schemas/omop/2.9}ObservableProperty'):
            params[p.get('{http://www.opengis.net/gml/3.2}id')] = p.find('{http://inspire.ec.europa.eu/schemas/omop/2.9}label').text
            
    return params        

def get_params(tree):
    """ Get parameters from response xml tree """
    retParams = []
    for el in tree.iter(tag='{http://www.opengis.net/om/2.0}observedProperty'):
        url = el.get('{http://www.w3.org/1999/xlink}href')
        #http://opendata.fmi.fi/meta?observableProperty=observation&param=t2m,ws_10min,wg_10min,wd_10min,rh,td,r_1h,ri_10min,snow_aws,p_sea,vis,n_man,wawa&language=eng
        params = re.findall(r"(?<=param=).*,.*(?=&)", url)[0].split(',')

        param_names = get_param_names(url)
        for p in params:
            retParams.append('{} ({})'.format(param_names[p], p))

    return retParams

def get_positions(tree):
    """ 
    Function to get times and coordinates from multipointcoverage answer
    """
    positions = []
    for element in tree.iter(tag='{http://www.opengis.net/gmlcov/1.0}positions'):
        pos = element.text.split()
        while len(pos) > 0:
            lat = float(pos.pop(0))
            lon = float(pos.pop(0))
            timestamp = int(pos.pop(0))
            positions.append([lat,lon,timestamp])

    return np.array(positions)


def save_data_to_csv(data, filename):
    """ 
    Function to save data to csv format as filename 
    """
    import pandas as pd
    selected_data = data[:, [0,1,2,4,6,14]]
    column_index = ['Latitude','Longitude','Timestamp','WindSpeed','WindDirection','CloudAmount']
    df = pd.DataFrame(selected_data, columns=column_index)
    df.Timestamp = df.Timestamp.astype(int) 
    #df.info()
    df['DateTime'] = pd.to_datetime(df.Timestamp, unit='s')
    header = ['Latitude','Longitude','DateTime','WindSpeed','WindDirection','CloudAmount']
    df.to_csv(filename+'.csv', index=False, columns = header)
    
    meanfilename = filename +'_mean.csv'
    df_mean = df.groupby(['DateTime'])['WindSpeed','WindDirection','CloudAmount'].mean()
    df_mean.to_csv(meanfilename, index=True)
    
    return df

def get_helsinki_bbox(filename):
    """ 
    Function to get helsinki bounding box from https://kartta.hel.fi
    """
    from owslib.wms import WebMapService                        
    
    wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.1.1')
    avoindata_contents = list(wms.contents)

    kantakartta_bbox = wms['avoindata:Kantakartta'].boundingBoxWGS84
    #print('\nlayer: Kantakartta BoundingBoxWGS85:\n{}'.format(kantakartta_bbox))
    # = (24.764732945863038, 59.90927898645569, 25.289558715996222, 60.322459856333225)

    img = wms.getmap(layers=['avoindata:Kantakartta'],
                    srs='EPSG:4326',
                    #bbox = (24.75, 60.00, 25.30, 60.35),
                    bbox = kantakartta_bbox,
                    size=(1000,1000),
                    format='image/png',
                    transparent=True)

    boundingbox = kantakartta_bbox
    
    out = open(filename, 'wb')
    out.write(img.read())
    out.close()

    return list(boundingbox)


def draw_weather_data(data_index, data, data_name, imagefilename, extentlist):
    import datetime as dt
    import matplotlib.pyplot as plt
    from matplotlib import colorbar, colors
  
    timestamp = data[0, 2] + 12*3600
    coords = data[(data[:, 2] == timestamp)][:, 0:2]

    value = data[(data[:, 2] == timestamp)][:, data_index]
    logitude = coords[:, 1]
    latitude = coords[:, 0]
    
    img = plt.imread(imagefilename)
    #plt.imshow(img, extent=[24.75, 25.30, 60.00, 60.35], alpha=0.5)
    plt.imshow(img, extent=extentlist, alpha=0.5)

    plt.scatter(logitude, latitude, c=value, cmap = 'coolwarm')
    cbar = plt.colorbar(cmap='coolwarm', orientation='vertical', ticklocation='auto', label = data_name)

    plt.title('Helsinki, {0}'.format(dt.datetime.fromtimestamp(timestamp)))
    plt.xlabel('Logitude')
    plt.ylabel('Latitude')
    plt.xlim(extentlist[0], extentlist[1])
    plt.ylim(extentlist[2], extentlist[3])
    
    plt.savefig(data_name +'_background.png')
    plt.show()
    

In [7]:
# ========= main ===========
#http://opendata.fmi.fi/wfs?service=WFS&version=2.0.0&request=getFeature&storedquery_id=fmi::observations::weather::multipointcoverage&place=kumpula&
#finland bbox = '19, 59, 30, 75'

import requests
import datetime as dt
import xml.etree.ElementTree as ET
import numpy as np


# User Input =========================================== 

# this function is for drawing weather data in Helsinki(download map as background image and get bounding box)
helsinki_bbox = get_helsinki_bbox('helsinki_map.png')
extent_list = [helsinki_bbox[0], helsinki_bbox[2], helsinki_bbox[1], helsinki_bbox[3]]
bbox_string_helsinki = str(helsinki_bbox)[1:-1]


# bounding box obtained from city map service site
bbox_string_helsinki = '24.764732945863038, 59.90927898645569, 25.289558715996222, 60.322459856333225'
bbox_string_tampere = '23.46654029784923, 61.40618508769914, 24.18867452905264, 61.85214253230824'
bbox_string_turku = '21.5, 60.19, 23.18, 60.91'
bbox_string_espoo = '24.48, 60.05, 24.85, 60.38'
bbox_string_vataa = '24.7456226944982,60.23704525911224,25.19371306316598,60.40131628464169'

# all input values must be string!
filename_to_save = ['helsinki_2020_5_1','tampere_2020_5_1', 'turku_2020_5_1']
bbox = [bbox_string_helsinki, bbox_string_tampere, bbox_string_turku]
start_time = '2020-05-01'
end_time = '2020-05-02'

#========================================================

df = [0 for _ in range(len(bbox))]

for i in range(len(bbox)):
    
    url = 'http://opendata.fmi.fi/wfs'

    parameter_input_data = {
        'request': 'getFeature',
        'storedquery_id': 'fmi::observations::weather::multipointcoverage',
        'bbox' : bbox[i],
        'starttime': start_time,
        'endtime': end_time
    }

    query_output = requests.get(url, params=parameter_input_data)
    tree = ET.ElementTree(ET.fromstring(query_output.content))

    positions = get_positions(tree)
    parameters = get_params(tree)

    d = []
    for element in tree.iter(tag='{http://www.opengis.net/gml/3.2}doubleOrNilReasonTupleList'):
        for item in element.text.strip().split("\n"):
            d.append(list(map(float, item.strip().split(' '))))

    data = np.append(positions, np.array(d), axis=1)

    #print('\nList of weather parameters:\n{0}'.format(parameters))
    print('\nPosisions array shape:', positions.shape)
    print('Weather data array shape:', np.array(d).shape)
    print('Combined positions and weather data shape:', data.shape)

    
    df[i] = save_data_to_csv(data, filename_to_save[i])

# output ================================================

# two differenct csv files created for each city
# 1. all observation-stations data is saved, vertically stacked (latitude and logitutde informations included)
# 2. average value is saved for each city (location information excluded, filename has city name) 

#========================================================


Posisions array shape: (5334, 3)
Weather data array shape: (5334, 13)
Combined positions and weather data shape: (5334, 16)


C:\Users\kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Posisions array shape: (1876, 3)
Weather data array shape: (1876, 13)
Combined positions and weather data shape: (1876, 16)

Posisions array shape: (2021, 3)
Weather data array shape: (2021, 13)
Combined positions and weather data shape: (2021, 16)


### Result of weather in Helsinki at 1.5.2020 3PM

There are many observation stations in Helsinki (bounding box includes other city's station located within boundary).

NOT all stations measure same features.

<table><tr><td style="text-align:left">Wind Speed (m/s) </td><td style="text-align:left">Wind Direction (0-360 degree)</td><td style="text-align:left">Cloud Amount (range: 0 - 8)</td></tr>
<tr><td><img src="wind_speed_backgroundX.png" alt="drawing" width=400/></td><td><img src="wind_direction_backgroundX.png" alt="drawing" width=400/></td><td><img src="cloud_amount_backgroundX.png" alt="drawing" width=400/></td></tr></table>

In [5]:
#background_image_filename = 'helsinki_map.png'
#draw_weather_data(4, data, 'wind_speed', background_image_filename, extent_list)
#draw_weather_data(6, data, 'wind_direction', background_image_filename, extent_list)
#draw_weather_data(14, data, 'cloud_amount', background_image_filename, extent_list)

df = df[1]
print('Wind direction(0-360 degree) max: {0} and min: {1}'.format(df.WindDirection.max(), df.WindDirection.min()))
print('Wind speed(m/s) max: {0} and min: {1}'.format(df.WindSpeed.max(), df.WindSpeed.min()))
print('Cloud amount(0-8) max: {0} and min: {1}'.format(df.CloudAmount.max(), df.CloudAmount.min()))
df.groupby(['Latitude','Longitude']).count()
print(df.isnull().values.any())

Wind direction(0-360 degree) max: 158.0 and min: 77.0
Wind speed(m/s) max: 9.5 and min: 1.9
Cloud amount(0-8) max: 8.0 and min: 0.0
True


In [137]:
#reference FMI github
#https://github.com/fmidev/opendata-resources/blob/master/examples/python/FMI_WFS2_getobs_multipointcoverage_example.ipynb3

#unused funcions below

In [122]:
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url='http://opendata.fmi.fi/wfs', version="2.0.0")

wfs.identification.title
wfs.identification.keywords
wfs.identification.type  # = 'wfs'
wfs.identification.service # = 'wfs'

wfs.get_schema
wfs.getfeature
wfs.getOperationByName

wfs.contents
wfs.operations

wfs['omso:PointTimeSeriesObservation'].title
wfs.getOperationByName('GetFeature').methods
wfs.getOperationByName('GetCapabilities').formatOptions

operation_list = []
#print(len(list(wfs.operations))) # = 31
for op in wfs.operations:
    if hasattr(op, 'name'):
        operation_list.append(op.name)

print(wfs.identification.keywords)
print('\nOperations with name: {0}, \n\nContents: {1}'.format(operation_list, list(wfs.contents)))

['Weather', 'Ocean', 'Radar', 'Observation', 'Forecast', 'Model', 'Hirlam', 'Radiation', 'Radioactivity', 'Airquality']

Operations with name: ['GetCapabilities', 'DescribeFeatureType', 'ListStoredQueries', 'DescribeStoredQueries', 'GetPropertyValue', 'GetFeature', 'version', 'ImplementsBasicWFS', 'ImplementsTransactionalWFS', 'ImplementsLockingWFS', 'KVPEncoding', 'XMLEncoding', 'SOAPEncoding', 'ImplementsInheritance', 'ImplementsRemoteResolve', 'ImplementsResultPaging', 'ImplementsStandardJoins', 'ImplementsSpatialJoins', 'ImplementsTemporalJoins', 'ImplementsFeatureVersioning', 'ManageStoredQueries', 'CountDefault', 'QueryExpressions'], 

Contents: ['BsWfs:BsWfsElement', 'avi:VerifiableMessage', 'ef:EnvironmentalMonitoringFacility', 'ef:EnvironmentalMonitoringNetwork', 'omso:GridSeriesObservation', 'omso:PointObservation', 'omso:PointTimeSeriesObservation', 'omso:ProfileObservation', 'omso:TrajectoryObservation']


In [134]:
def get_observation_stations_list():
    import pandas as pd
    table = pd.read_html('https://en.ilmatieteenlaitos.fi/observation-stations')
    df = table[0]
    df.to_csv('observation_stations.csv', index=False)

In [156]:
def get_list_storedQuery(operation_name):
    import xml.etree.ElementTree as ET
    import requests
    
    url = 'http://opendata.fmi.fi/wfs'
    
    list_childquery_tag = []
    list_storedquery =[]
    
    operation = requests.get(url, params = {'request':operation_name}) 
    #http://opendata.fmi.fi/wfs?request=DescribeFeatureType
    #http://opendata.fmi.fi/wfs?service=WFS&version=2.0.0&request=listStoredQueries&
    #http://opendata.fmi.fi/wfs?request=GetCapabilities
    query = ET.fromstring(operation.content)
    print(query.tag) #, query.attrib)
    
    for childquery in query:
        if childquery.tag not in list_childquery_tag: 
            list_childquery_tag.append(childquery.tag)
            #print(childquery.tag)
            
    length_childquery = len(list_childquery_tag)
    print('Number of unique childquery: {0}\n'.format(length_childquery))
    
    for item in query.iter(list_childquery_tag[0]):
        x = dict(item.attrib)
        for key in x: 
            list_storedquery.append(item.get(key))
            
    return list_storedquery


# available operations for this method from the wfs.operations
# GetCapabilities- 5, DescribeFeatureType- 2, ListStoredQueries- 1, DescribeStoredQueries- 1

storedQueryList = get_list_storedQuery('ListStoredQueries')
get_list_storedQuery('DescribeStoredQueries')

{http://www.opengis.net/wfs/2.0}ListStoredQueriesResponse
Number of unique childquery: 1

{http://www.opengis.net/wfs/2.0}DescribeStoredQueriesResponse
Number of unique childquery: 1



['fmi::avi::observations::finland::iwxxm',
 'fmi::avi::observations::finland::latest::iwxxm',
 'fmi::avi::observations::iwxxm',
 'fmi::avi::observations::latest::iwxxm',
 'fmi::ef::networks',
 'fmi::ef::stations',
 'fmi::forecast::climatology::scenario::grid',
 'fmi::forecast::enfuser::airquality::helsinki-metropolitan::grid',
 'fmi::forecast::harmonie::hybrid::grid',
 'fmi::forecast::harmonie::hybrid::point::multipointcoverage',
 'fmi::forecast::harmonie::hybrid::point::simple',
 'fmi::forecast::harmonie::hybrid::point::timevaluepair',
 'fmi::forecast::harmonie::pressure::grid',
 'fmi::forecast::harmonie::pressure::point::multipointcoverage',
 'fmi::forecast::harmonie::pressure::point::simple',
 'fmi::forecast::harmonie::pressure::point::timevaluepair',
 'fmi::forecast::harmonie::surface::grid',
 'fmi::forecast::harmonie::surface::point::multipointcoverage',
 'fmi::forecast::harmonie::surface::point::simple',
 'fmi::forecast::harmonie::surface::point::timevaluepair',
 'fmi::forecast::

In [136]:
def get_bbox(filename):
    
    from owslib.wms import WebMapService 
    import io                            
    import numpy as np                   
    import matplotlib.pyplot as plt      
    from PIL import Image
    import matplotlib.image as mpimg

    wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.1.1')

    print('https://kartta.hel.fi/ws/geoserver/avoindata/wms identification:\nType: {0}\nVersion: {1}\nTitle: {2}\nAbstract: {3}'.format(wms.identification.type, wms.identification.version, wms.identification.title, wms.identification.abstract))

    avoindata_contents = list(wms.contents)

    ortoilmakuva = 'avoindata:Ortoilmakuva_2019_5cm'
    print('\nLayer of Ortoilmakuva_2019_5cm:\n')
    print('\nTitle: {0}\nqueryable: {1}\nopaque: {2}'.format(wms[ortoilmakuva].title, wms[ortoilmakuva].queryable, wms[ortoilmakuva].opaque))
    print('\nCRS options:{}'.format(wms[ortoilmakuva].crsOptions))

    ortoilmakuva_2019_5cm_bbox = wms['avoindata:Ortoilmakuva_2019_5cm'].boundingBoxWGS84
    print('\nlayer: Ortoilmakuva_2019_5cm BoundingBoxWGS85:\n{}'.format(ortoilmakuva_2019_5cm_bbox))
    # = (24.764732945863038, 60.0080107531311, 25.289558715996222, 60.322459856333225)

    kantakartta_bbox = wms['avoindata:Kantakartta'].boundingBoxWGS84
    print('\nlayer: Kantakartta BoundingBoxWGS85:\n{}'.format(kantakartta_bbox))
    # = (24.764732945863038, 59.90927898645569, 25.289558715996222, 60.322459856333225)

    print('\nOperations name:')
    for op in wms.operations:
        print('-', op.name)

    print('GetMap methods:', wms.getOperationByName('GetMap').methods)
    print(wms.getOperationByName('GetMap').formatOptions) 

    img = wms.getmap(layers=['avoindata:Kantakartta'],
                    srs='EPSG:4326',
                    #bbox = (24.75, 60.00, 25.30, 60.35),
                    bbox = kantakartta_bbox,
                    size=(1000,1000),
                    format='image/png',
                    transparent=True)

    boundingbox = kantakartta_bbox
    out = open(filename, 'wb')
    out.write(img.read())
    out.close()

    return boundingbox

#out = open('helsinki_kantakartta.png', 'wb')
#out.write(img.read())
#out.close()

#image = Image.open('helsinki_kartta_ilma.jpg')
#image.show()
#image.save('helsinki_map.jpg')

#img = mpimg.imread('helsinki_kartta_ilma.png')
#plt.imshow(img)
#imgplot = plt.imshow(img)



In [117]:
def get_bbox_tampere(filename):
    
    #https://georaster.tampere.fi/geoserver/georaster/wms?service=WMS&version=1.3.0&request=GetCapabilities
    from owslib.wms import WebMapService                      
    import numpy as np                   
    import matplotlib.image as mpimg

    wms = WebMapService('https://georaster.tampere.fi/geoserver/georaster/wms', version='1.3.0')

    #print('https://georaster.tampere.fi/geoserver/georaster/wms identification:\nType: {0}\nVersion: {1}\nTitle: {2}\nAbstract: {3}'.format(wms.identification.type, wms.identification.version, wms.identification.title, wms.identification.abstract))

    contents = list(wms.contents) #list of layer
    
    #print('\nOperations name:')
    #for op in wms.operations:
    #    print('-', op.name)
        
    #print('\nGetMap methods: {}'.format(wms.getOperationByName('GetMap').methods))
    
    kantakartta = 'kantakartta_EPSG_3067'
    #print('\nLayer of kantakartta_EPSG_3067:')
    #print('Title: {0}\nqueryable: {1}, opaque: {2}'.format(wms[kantakartta].title, wms[kantakartta].queryable, wms[kantakartta].opaque))
    #print('\nCRS options:{}'.format(wms[kantakartta].crsOptions))
    #print('name of layer: {}'.format(wms[kantakartta].name))
    
    kantakartta_bbox = wms[kantakartta].boundingBox
    print('\nkantakartta_EPSG_3067 CRS:84 BoundingBox:\n{}'.format(kantakartta_bbox))
    #= (23.46654029784923, 61.40618508769914, 24.18867452905264, 61.85214253230824, 'CRS:84')
    bbox_kanta = kantakartta_bbox[:-1]
    
    img = wms.getmap(layers=[kantakartta],
                    srs='CRS:84',
                    bbox = bbox_kanta,
                    size=(1000,1000),
                    format='image/png',
                    transparent=True)
    url = 'https://georaster.tampere.fi/geoserver/georaster/wms?service=WMS&version=1.3.0&request=GetMap&layers=kantakartta_EPSG_3067&CRS=crs:84&BBOX=23.46654029784923,61.40618508769914,24.18867452905264,61.85214253230824&WIDTH=1000&HEIGHT=1000&FORMAT=image/png'
    
    ''' 
    <Name>kantakartta_EPSG_3067</Name>
    <Title>Tampereen kantakartta (MV, TM35)</Title>
    BBOX=minx,miny,maxx,maxy
    <EX_GeographicBoundingBox>
    <westBoundLongitude>23.46654029784923</westBoundLongitude>
    <eastBoundLongitude>24.18867452905264</eastBoundLongitude>
    <southBoundLatitude>61.40618508769914</southBoundLatitude>
    <northBoundLatitude>61.85214253230824</northBoundLatitude>
    </EX_GeographicBoundingBox>
    <BoundingBox CRS="CRS:84" minx="23.46654029784923" miny="61.40618508769914" maxx="24.18867452905264" maxy="61.85214253230824"/>
    '''

    out = open(filename, 'wb')
    out.write(img.read())
    out.close()

    return list(bbox_kanta)

#get_bbox_tampere('tampere.png')
tampere_bbox = get_bbox_tampere('tampere_map.png')
bbox_string_tampere = str(tampere_bbox)[1:-1]


In [115]:
def get_bbox_turku(filename):
    
    #https://opaskartta.turku.fi/TeklaOGCWeb/WMS.ashx?request=getCapabilities
    from owslib.wms import WebMapService                          

    wms = WebMapService('https://opaskartta.turku.fi/TeklaOGCWeb/WMS.ashx?', version='1.1.1')

    print('https://opaskartta.turku.fi/TeklaOGCWeb/WMS.ashx? identification:\nType: {0}\nVersion: {1}\nTitle: {2}\nAbstract: {3}'.format(wms.identification.type, wms.identification.version, wms.identification.title, wms.identification.abstract))

    contents = list(wms.contents) #list of layer
    
    print('\nOperations name:')
    for op in wms.operations:
        print('-', op.name)
        
    print('\nGetMap methods: {}'.format(wms.getOperationByName('GetMap').methods))
   
    
    kantakartta = 'Turun kantakartta 1:1000'
    print('\nLayer of Turun kantakartta 1:1000:')
    print('Title: {0}\nqueryable: {1}, opaque: {2}'.format(wms[kantakartta].title, wms[kantakartta].queryable, wms[kantakartta].opaque))
    print('name of layer: {}'.format(wms[kantakartta].name))
    
    kantakartta_bbox = wms[kantakartta].boundingBox
    print('Turun kantakartta 1:1000 EPSG:3877 BoundingBox:\n{}'.format(kantakartta_bbox))
    
    # = <LatLonBoundingBox minx="21.50" miny="60.19" maxx="23.18" maxy="60.91"/>
    bbox_kanta = (21.50, 60.19, 23.18, 60.91)
    
    img = wms.getmap(layers=[kantakartta],
                    srs = "EPSG:4326",
                    bbox = bbox_kanta,
                    size=(1000,1000),
                    format='image/png',
                    transparent=True)
    
    url = 'https://opaskartta.turku.fi/TeklaOGCWeb/WMS.ashx?request=GetMap&layers=Turun kantakartta 1:1000&SRS=EPSG:4326&BBOX=21.50,60.19,23.18,60.91&WIDTH=1000&HEIGHT=1000&FORMAT=image/png'
    
    out = open(filename, 'wb')
    out.write(img.read())
    out.close()

    return list(bbox_kanta)

''' 
<Layer>
<Name>Turun kantakartta 1:1000</Name>
<Title>Turun kantakartta 1:1000</Title>
<Abstract>Turun kantakartta 1:1000</Abstract>
<KeywordList>
<Keyword>Kantakartta 1:1000 rasteri</Keyword>
</KeywordList>
<LatLonBoundingBox minx="21.50" miny="60.19" maxx="23.18" maxy="60.91"/>
<BoundingBox SRS="EPSG:3877" minx="23417000.00" miny="6676000.00" maxx="23510000.00" maxy="6756000.00"/>
<BoundingBox SRS="EPSG:2391" minx="1528146.89" miny="6675315.67" maxx="1618677.54" maxy="6758109.16"/>
<BoundingBox SRS="EPSG:4326" minx="21.50" miny="60.19" maxx="23.18" maxy="60.91"/>
<BoundingBox SRS="EPSG:3857" minx="2393808.37" miny="8441796.05" maxx="2580867.06" maxy="8606257.63"/>
<BoundingBox SRS="EPSG:3386" minx="694541.79" miny="6680376.31" maxx="781256.31" maxy="6767267.14"/>
<BoundingBox SRS="EPSG:2392" minx="2361712.96" miny="6677818.24" maxx="2455926.91" maxy="6756401.89"/>
<BoundingBox SRS="EPSG:2393" minx="3195471.12" miny="6687887.35" maxx="3293240.04" maxy="6762143.37"/>
<BoundingBox SRS="EPSG:2394" minx="4029655.12" miny="6705536.22" maxx="4130853.33" maxy="6775340.17"/>
<BoundingBox SRS="EPSG:3387" minx="4864504.38" miny="6730787.46" maxx="4969006.92" maxy="6796007.09"/>
<BoundingBox SRS="EPSG:3067" minx="195424.00" miny="6685079.19" maxx="293153.30" maxy="6759305.88"/>
<BoundingBox SRS="EPSG:3126" minx="638902.82" miny="6677693.85" maxx="726903.74" maxy="6763231.29"/>
<BoundingBox SRS="EPSG:3127" minx="583437.84" miny="6676009.95" maxx="672710.76" maxy="6760181.69"/>
<BoundingBox SRS="EPSG:3128" minx="527960.02" miny="6675166.40" maxx="618489.98" maxy="6757960.11"/>
<BoundingBox SRS="EPSG:3129" minx="472477.89" miny="6675163.09" maxx="564250.15" maxy="6756566.27"/>
<BoundingBox SRS="EPSG:3130" minx="417000.00" miny="6676000.00" maxx="510000.00" maxy="6756000.00"/>
<BoundingBox SRS="EPSG:3131" minx="361534.89" miny="6677677.27" maxx="455748.25" maxy="6756261.22"/>
<BoundingBox SRS="EPSG:3132" minx="306091.14" miny="6680195.14" maxx="401503.60" maxy="6757349.96"/>
<BoundingBox SRS="EPSG:3133" minx="250677.33" miny="6683553.98" maxx="347274.78" maxy="6759266.37"/>
<BoundingBox SRS="EPSG:3134" minx="195302.12" miny="6687754.29" maxx="293070.53" maxy="6762010.68"/>
<BoundingBox SRS="EPSG:3135" minx="139974.20" miny="6692796.68" maxx="238899.62" maxy="6765583.25"/>
<BoundingBox SRS="EPSG:3136" minx="84702.35" miny="6698681.87" maxx="184770.90" maxy="6769984.54"/>
<BoundingBox SRS="EPSG:3137" minx="29495.42" miny="6705410.70" maxx="130693.22" maxy="6775215.08"/>
<ScaleHint min="0.1768" max="90.5097"/>
</Layer>
'''
    
#get_bbox_turku('turku.png')
bye = 0